In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./mnist/data/', one_hot = True)
X = tf.placeholder(tf.float32, [None, 28* 28])
Z = tf.placeholder(tf.float32, [None, 128]) # Noise Dimension = 128

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [0]:
#Generator Network
G_W1 = tf.Variable(tf.random_normal([128,256], stddev=0.01))
G_W2 = tf.Variable(tf.random_normal([256,28* 28], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([256]))
G_b2 = tf.Variable(tf.zeros([28 * 28]))

def generator(noise_z):# 128 -> 256 -> 28 * 28
  hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
  return output
  

In [0]:
# D-Network(Hidden Node = 256)
D_W1 = tf.Variable(tf.random_normal([28*28, 256], stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([256,1], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([256]))
D_b2 = tf.Variable(tf.zeros([1]))

def discriminator(inputs): #28 * 28 -> 256 -> 1
  hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
  return output
  

In [0]:
# Generation , Loss , Optimization
G = generator(Z)

loss_D = -tf.reduce_mean(tf.log(discriminator(X)) + tf.log(1- discriminator(G)))
loss_G = -tf.reduce_mean(tf.log(discriminator(G)))

train_D = tf.train.AdamOptimizer(learning_rate = 0.0002).minimize(loss_D, var_list=[D_W1, D_b1, D_W2, D_b2])
train_G = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(loss_G, var_list=[G_W1, G_b1, G_W2, G_b2])

In [12]:
!mkdir samples_ex
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  noise_test = np.random.normal(size = (10,128))# 10 = Test Sample Size, 128 = Noise Dimension
  for epoch in range(200):
    print(epoch)
    for i in range(int(mnist.train.num_examples / 100)):# 100 = Batch Size
      batch_xs, _ = mnist.train.next_batch(100)
      noise = np.random.normal(size = (100,128))
      
      sess.run(train_D, feed_dict={X: batch_xs, Z: noise})
      sess.run(train_G, feed_dict={Z: noise})
    if epoch == 0 or (epoch + 1) % 10 == 0: # 10 = Saving Period
        samples = sess.run(G, feed_dict={Z: noise_test})

        fig, ax = plt.subplots(1, 10, figsize=(10, 1))
        for i in range(10):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        plt.savefig('samples_ex/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
      

mkdir: cannot create directory ‘samples_ex’: File exists
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
